In [ ]:
!pip install pandas numpy spacy scikit-learn tensorflow datasets spacy_langdetect
!pip install lckr_jupyterlab_variableinspector

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import spacy
from spacy_langdetect import LanguageDetector
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder


In [ ]:
ds = load_dataset("maharshipandya/spotify-tracks-dataset")

In [ ]:
#get train and test datasets
df_train = pd.DataFrame(ds['train'])
df_test = pd.DataFrame(ds['train'])

In [ ]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [ ]:
#drop useless columns
df_train.drop(columns=['Unnamed: 0', 'track_id'], inplace=True)
df_test.drop(columns=['Unnamed: 0', 'track_id'], inplace=True)

In [ ]:
#get prediction columns and drop from tables
scalar = MinMaxScaler()
target_train = df_train['popularity']
target_test = df_test['popularity']
df_train.drop(columns=['popularity'], inplace=True)
df_test.drop(columns=['popularity'], inplace=True)

In [ ]:
#scale duration
df_train['duration_ms'] = scalar.fit_transform(df_train[['duration_ms']]).flatten()
df_test['duration_ms'] = scalar.fit_transform(df_test[['duration_ms']]).flatten()

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
#one hot encode explicit and track genre
temp_train = pd.DataFrame(encoder.fit_transform(df_train[['explicit']]), columns=encoder.get_feature_names_out(['explicit']))
temp2_train = pd.DataFrame(encoder.fit_transform(df_train[['track_genre']]), columns=encoder.get_feature_names_out(['track_genre']))


temp3_test = pd.DataFrame(encoder.fit_transform(df_test[['explicit']]), columns=encoder.get_feature_names_out(['explicit']))
temp4_test = pd.DataFrame(encoder.fit_transform(df_test[['track_genre']]), columns=encoder.get_feature_names_out(['track_genre']))

In [ ]:
df_train = pd.concat([df_train,temp_train,temp2_train], axis=1)
df_test = pd.concat([df_test, temp3_test, temp4_test], axis=1)
df_train.drop(columns=['explicit', 'track_genre'], inplace=True)
df_test.drop(columns=['explicit','track_genre'], inplace=True)


In [ ]:
#one hot encode key of song 
temp_train_key = pd.DataFrame(encoder.fit_transform(df_train[['key']]), columns=encoder.get_feature_names_out(['key']))
df_train = pd.concat([df_train,temp_train_key], axis=1)
df_train.drop(columns=['key'], inplace=True)


temp_test_key = pd.DataFrame(encoder.fit_transform(df_test[['key']]), columns=encoder.get_feature_names_out(['key']))
df_test = pd.concat([df_test, temp_test_key], axis=1)
df_test.drop(columns=['key'], inplace=True)

In [ ]:
#scale loudness
df_train['loudness'] = scalar.fit_transform(df_train[['loudness']]).flatten()
df_test['loudness'] = scalar.fit_transform(df_test[['loudness']]).flatten()

#one hot encode time signature
temp_train_tim_sig = pd.DataFrame(encoder.fit_transform(df_train[['time_signature']]), columns=encoder.get_feature_names_out(['time_signature']))
temp_test_tim_sig = pd.DataFrame(encoder.fit_transform(df_test[['time_signature']]), columns=encoder.get_feature_names_out(['time_signature']))

df_train=pd.concat([df_train,temp_train_tim_sig], axis=1)
df_train.drop(columns=['time_signature'], inplace=True)

df_test = pd.concat([df_test,temp_test_tim_sig],axis=1)
df_test.drop(columns=['time_signature'], inplace=True)

In [ ]:
# nlp = spacy.load("en_core_web_sm")
# @spacy.Language.factory("language_detector")

# def create_lang_detector(nlp, name):
#     return LanguageDetector()

# nlp.add_pipe('language_detector', last=True)

# def detect(text):
#     doc = nlp(text)
#     return doc._.language['language']
# df['album_lang'] = df['album_name'].apply(detect)